In [ ]:
import requests
from scrapy.http import TextResponse
import urllib
from pathlib import Path
import os

os.mkdir('svg')
categories = [
"cardiovascular",
"central nervous system",
"anti-infective",
"gastrointestinal",
"anti-inflammatory",
"dermatological",
"hematologic",
"lipid regulating",
"reproductive control",
"respiratory system",
"urological",
"antineoplastic"]
def drugCategory(category):
    res1 = requests.get('https://www.drugbank.ca/categories?utf8=%E2%9C%93&approved=0&nutraceutical=0&illicit=0&investigational=0&withdrawn=0&experimental=0&us=0&ca=0&eu=0&commit=Apply+Filter&q%5Btitle%5D='+category+'&q%5Bdescription%5D=&q%5Bdrug_count%5D=&q%5Btarget_count%5D=')
    response1 = TextResponse(res1.url, body=res1.text, encoding='utf-8')
    l = []
    for href in response1.css('a::attr(href)'):
        if "/categories/" in href.extract():
            l.append("https://www.drugbank.ca"+href.extract())
    l = list(set(l))
    dlinks = []
    dlinks1 = []
    for link in l:
        res2 = requests.get(link)
        response2 = TextResponse(res2.url, body=res2.text, encoding='utf-8')
        d=[]
        for druglink in response2.css('a::attr(href)'):
            if "/drugs/" in druglink.extract():
                d.append("https://www.drugbank.ca"+druglink.extract())
                dlinks1.append("https://www.drugbank.ca"+druglink.extract())
        dlinks.append(list(set(d)))
    return list(set(dlinks1))
def storeImages(urls, folder):
    os.mkdir("svg/"+folder)
    for i in urls:
        img = i.split("/")[-1]
        imageLink = "https://www.drugbank.ca/structures/"+img+"/image.svg"
        urllib.request.urlretrieve(imageLink, folder+"/"+img+".svg")
for category in categories:
    Path(category).mkdir(parents=True, exist_ok=True)
    drugLinks = drugCategory(category)
    print(category, len(drugLinks))
    storeImages(drugLinks, category)